### Задание 3 - 10 баллов

- Загрузить набор данных [Spam Or Not Spam](https://www.kaggle.com/datasets/ozlerhakan/spam-or-not-spam-dataset) (или любой другой, какой вам нравится)
- Обучить модели и сравнить различные способы векторизации с помощью внутренней оценки (intrinsic):
  - Word2Vec SkipGram / CBOW (параметр sg в `gensim.models.word2vec.Word2Vec`) - **3 балла**
  - fastText (можно взять в gensim, или в fasttext как на семинаре) - **2 балла**
- Обучить на полученных векторах модели LogisticRegression и сравнить качество на отложенной выборке - **2 балла**

- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **2 балла**

- Соблюден code style на уровне pep8 и [On writing clean Jupyter notebooks](https://ploomber.io/blog/clean-nbs/)  - **1 балл**
 
Примечания:

- Для получения более качественных эмбеддингов стоит предварительно сделать предобработку корпуса - отсеять стоп-слова, провести нормализацию и тп. Предобработка рассматривалась в первой лекции/семинаре
- В данном случае под intrinsic оценкой подразумевается просто использование методов `most_similar`, `doesnt_match`. Однако, если есть желание, можно измерить косинусное расстояние между отдельными парами слов и проверить, есть ли корреляция с корпусами для intrinsic-оценки, которые обсуждались на семинаре


## Все библиотеки и константы



Импортируем, добавляем константы, функции

In [1]:
import os

import nltk
import numpy as np
import pandas as pd

from gensim.models.word2vec import Word2Vec
from gensim.models import FastText
import warnings
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import spacy

warnings.simplefilter(action='ignore')

SEED = 566
# data dir
DATA_DIR = "../data"

DATA = "../data/spam_or_not_spam.csv"


def classification_report_pd(y_test, y_pred):
    report = pd.DataFrame(classification_report(y_true=y_test, y_pred=y_pred, output_dict=True)).transpose()
    report.support = report.support.astype(int)
    report.loc['accuracy', 'support'] = report.loc['macro avg', 'support']
    report.loc['accuracy', 'precision'] = np.nan
    report.loc['accuracy', 'recall'] = np.nan
    return report


In [2]:
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/toharhymes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/toharhymes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Загрузка данных

Disclaimer: Я сначала попробовал поработать с данными из первого ДЗ -- предсказывать топики, но так как там классов (топиков) много, то все очень плохо обучалось, решил взять из второго ДЗ на англоязычных текстах spam-not spam.

(не забудьте раскоментить):

In [4]:
# !kaggle datasets download -d ozlerhakan/spam-or-not-spam-dataset
# !mv ./spam-or-not-spam-dataset.zip ../data/
# !unzip ../data/spam-or-not-spam-dataset.zip
# !mv ./spam_or_not_spam.csv ../data/
# !python -m spacy download en_core_web_sm

#### Читаем датасет

In [5]:
data = pd.read_csv(DATA)
data = data[(~data['email'].isna()) & (data['label'].isin([0, 1]))]
print('Shape:', data.shape)
print('Classes:', data.label.value_counts())
data.head()

Shape: (2999, 2)
Classes: label
0    2500
1     499
Name: count, dtype: int64


,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


# Предобработка

Проведем предобработку данных, используя методы, похожие первой лабораторной:

In [6]:
NLP = spacy.load("en_core_web_sm")


def check_token(token):
    return not (token.is_stop
                or token.is_punct
                or token.is_digit
                or token.like_email
                or token.like_num)  # like num в данном случае делать не обязательно (тут уже заменены все NUMBER), но сделал это для общности -- на будущее


def tokenize_clean(text):
    return [token.lemma_.lower() for token in NLP(text) if check_token(token)]

In [7]:
%%time
data['email_tokenized'] = data['email'].apply(tokenize_clean)
data.head()

CPU times: user 1min, sys: 467 ms, total: 1min
Wall time: 1min


,email,label,email_tokenized
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,"[ , date, d, number, aug, number, number, numb..."
1,martin a posted tassos papadopoulos the greek ...,0,"[martin, post, tassos, papadopoulo, greek, scu..."
2,man threatens explosion in moscow thursday aug...,0,"[man, threaten, explosion, moscow, thursday, a..."
3,klez the virus that won t die already the most...,0,"[klez, virus, win, t, die, prolific, virus, kl..."
4,in adding cream to spaghetti carbonara which ...,0,"[ , add, cream, spaghetti, carbonara, effect, ..."


### Split data

In [8]:
X, y = data.email_tokenized, data.label

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=SEED,
                                                    stratify=y)

display(y_train.value_counts())
display(y_test.value_counts())

label
0    2000
1     399
Name: count, dtype: int64

label
0    500
1    100
Name: count, dtype: int64

## Учим Эмбеддинги

### Word2Vec SkipGram / CBOW

Попробую и skip=gram и CBOW, и погляжу, что лучше обучилось (все остальные параметры оставлю одинаковыми)

In [9]:
%%time

model_skipgram = Word2Vec(
    sentences=X_train,
    vector_size=300,  # default = 100
    window=5,  # default = 5
    min_count=10,
    sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0,  #  1 - hierarchical softmax. 0, and negative>0 - negative sampling.
    negative=5,  # If > 0 -- negative sampling will be used
    epochs=40,  # epochs over the corpus
    seed=SEED,
    workers=16
)

CPU times: user 2min 30s, sys: 374 ms, total: 2min 30s
Wall time: 13.2 s


In [10]:
%%time
model_cbow = Word2Vec(
    sentences=X_train,
    vector_size=300,  # default = 100
    window=5,  # default = 5
    min_count=10,
    sg=0,  # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0,  #  1 - hierarchical softmax. 0, and negative>0 - negative sampling.
    negative=5,  # If > 0 -- negative sampling will be used
    epochs=40,  # epochs over the corpus
    seed=SEED,
    workers=16
)

CPU times: user 27.4 s, sys: 184 ms, total: 27.6 s
Wall time: 4.98 s


In [11]:
model_skipgram.wv.most_similar(positive=['october'], topn=5), model_cbow.wv.most_similar(positive=['october'], topn=5)

([('gnat', 0.46857908368110657),
  ('tuesday', 0.4551939368247986),
  ('rafael', 0.44891849160194397),
  ('ziggy', 0.3902868926525116),
  ('porter', 0.3701557517051697)],
 [('august', 0.7366386651992798),
  ('tuesday', 0.7045066952705383),
  ('december', 0.690962016582489),
  ('september', 0.649682343006134),
  ('february', 0.6377255916595459)])

Cbow выглядит получше -- просто  месяцы выдает, а не что-то непонятное.

In [12]:
model_skipgram.wv.most_similar(positive=['girl'], topn=5), model_cbow.wv.most_similar(positive=['girl'], topn=5)

([('teen', 0.43621835112571716),
  ('sexy', 0.43331652879714966),
  ('xxx', 0.4104221761226654),
  ('orlando', 0.39990732073783875),
  ('servant', 0.38169562816619873)],
 [('teen', 0.6404370665550232),
  ('xxx', 0.5881308317184448),
  ('cum', 0.5460056066513062),
  ('sexy', 0.5423579216003418),
  ('fucking', 0.5155233144760132)])

Из-за специфики текстов, немного специфичное окружение слова "girl", ну а что поделать.

In [13]:
model_skipgram.wv.most_similar(positive=['money'], topn=5), model_cbow.wv.most_similar(positive=['money'], topn=5)

([('upfront', 0.4077613651752472),
  ('consignment', 0.3211933672428131),
  ('mega', 0.3172152638435364),
  ('investment', 0.3057568371295929),
  ('agm', 0.3055465817451477)],
 [('investment', 0.4264647364616394),
  ('incredibly', 0.37271201610565186),
  ('cash', 0.3530132472515106),
  ('pocket', 0.3332630395889282),
  ('estate', 0.3321537971496582)])

Кажется, что тоже cbow получше.

In [14]:
model_skipgram.wv.most_similar(positive=['money'], negative=["win"])[:6], model_cbow.wv.most_similar(
    positive=['money'], negative=["win"])[:6]

([('upfront', 0.23783999681472778),
  ('acquisition', 0.19944292306900024),
  ('payment', 0.17025168240070343),
  ('la', 0.16988345980644226),
  ('government', 0.16855306923389435),
  ('devote', 0.16733218729496002)],
 [('plus', 0.3156777620315552),
  ('pocket', 0.2928771674633026),
  ('payment', 0.2808143198490143),
  ('p', 0.27176758646965027),
  ('thailand', 0.2545802593231201),
  ('assist', 0.24979622662067413)])

In [15]:
model_skipgram.wv.most_similar(positive=['document'], negative=["money"])[:6], model_cbow.wv.most_similar(
    positive=['document'],
    negative=["money"])[
                                                                               :6]

([('february', 0.19939151406288147),
  ('template', 0.17645825445652008),
  ('inter', 0.17278490960597992),
  ('paragraph', 0.16987238824367523),
  ('assertion', 0.1673910915851593),
  ('namespace', 0.16625744104385376)],
 [('cite', 0.4572882056236267),
  ('relevant', 0.4489752948284149),
  ('hebrew', 0.4096534252166748),
  ('transmit', 0.40616336464881897),
  ('assessment', 0.3861862123012543),
  ('inter', 0.38413017988204956)])

In [16]:
model_skipgram.wv.most_similar(positive=['study', "school"])[:6], model_cbow.wv.most_similar(
    positive=['study', "school"])[:6]

([('undergraduate', 0.46846500039100647),
  ('graduate', 0.4104095995426178),
  ('appoint', 0.3900448977947235),
  ('college', 0.35854992270469666),
  ('phd', 0.3525552451610565),
  ('forbe', 0.349665105342865)],
 [('undergraduate', 0.5715292096138),
  ('graduate', 0.5712165832519531),
  ('education', 0.5570700168609619),
  ('degree', 0.5340072512626648),
  ('science', 0.533780574798584),
  ('academic', 0.48827579617500305)])

В последнем случае очень похоже, классно. В целом хороши оба, но на глазок, cbow в чем-то лучше.

### Fasttext

Сама библиотека fasttext отказывается вставать, поэтому воспользуемся fasttext'ом из gensim'a

In [17]:
%%time
model_fasttext = FastText(vector_size=300,
                          window=5,
                          min_count=1,
                          sentences=X_train,
                          epochs=40,
                          seed=SEED,
                          workers=16)

CPU times: user 6min 34s, sys: 2.06 s, total: 6min 36s
Wall time: 44 s


In [18]:
model_fasttext.wv.most_similar(positive=['october'], topn=5)

[('november', 0.8149967789649963),
 ('octobre', 0.7829334139823914),
 ('sober', 0.7816057205200195),
 ('december', 0.7784882187843323),
 ('bomber', 0.7690507173538208)]

In [19]:
model_fasttext.wv.most_similar(positive=['girl'], topn=5)

[('peegirl', 0.8596227169036865),
 ('giro', 0.8107101321220398),
 ('peegirls', 0.7703859210014343),
 ('gimmicky', 0.6342937350273132),
 ('schoolgirl', 0.6258540749549866)]

In [20]:
model_fasttext.wv.most_similar(positive=['money'], topn=5)

[('easymoney', 0.9122400879859924),
 ('foundmoney', 0.9036589860916138),
 ('mooney', 0.9009780883789062),
 ('honey', 0.8917720913887024),
 ('moneystarte', 0.8832082748413086)]

In [21]:
model_fasttext.wv.most_similar(positive=['money'], negative=["win"])[:6]

[('foundmoney', 0.6533924341201782),
 ('honey', 0.6199032068252563),
 ('easymoney', 0.6172727346420288),
 ('mooney', 0.6161479353904724),
 ('moneystarte', 0.6013731956481934),
 ('clooney', 0.5991310477256775)]

In [22]:
model_fasttext.wv.most_similar(positive=['document'], negative=["money"])[:6]

[('documentary', 0.6510372757911682),
 ('documentum', 0.6248524188995361),
 ('entanglement', 0.5791913866996765),
 ('notamment', 0.5538952946662903),
 ('environnementaux', 0.5495142340660095),
 ('clements', 0.54843670129776)]

In [23]:
model_fasttext.wv.most_similar(positive=['study', "school"])[:6],

([('schoolgirls', 0.7406686544418335),
  ('schoolchildren', 0.7239114046096802),
  ('schoolmate', 0.7111110091209412),
  ('scholar', 0.6989538073539734),
  ('preschool', 0.6978604197502136),
  ('schoolgirl', 0.6958659291267395)],)

Тут в основном производятся разные формы слова, а не синонимы ну -- тоже интересно. Посмотрим, что будет лучше.
Для начала получим векторы этих текстов:

In [24]:
def text_embedding(text, model):
    vectors = [model.wv[word] if word in model.wv else np.empty(300) for word in text]
    embedding = np.nanmean(vectors, axis=0)
    return embedding


def corpus_embedding(corpus, model):
    # Get the vector for each word and average them
    embeddings = [text_embedding(text, model) for text in corpus]
    return embeddings

In [25]:
%%time
embedding_models = {'cbow': model_cbow,
                    'skipgram': model_skipgram,
                    'fasttext': model_fasttext}
X_trains = dict()
X_tests = dict()
for model_name in embedding_models:
    print(model_name)
    X_trains[model_name] = np.array(corpus_embedding(X_train, embedding_models[model_name]))
    X_tests[model_name] = np.array(corpus_embedding(X_test, embedding_models[model_name]))
    print(X_trains[model_name].shape, X_tests[model_name].shape)

cbow
(2399, 300) (600, 300)
skipgram
(2399, 300) (600, 300)
fasttext
(2399, 300) (600, 300)
CPU times: user 2.43 s, sys: 683 ms, total: 3.11 s
Wall time: 2.27 s


## Предсказание

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [29]:
grid = {"lr__C": np.linspace(0.1, 1, 10),
        "lr__penalty": ("l1", "l2"),
        "lr__random_state": [SEED, ]}

pipe = Pipeline(
    steps=[
        ('lr', LogisticRegression())
    ]
)

grid_search = GridSearchCV(
    pipe,
    param_grid=grid,
    n_jobs=-1,
    verbose=1,
    cv=5,
    scoring='accuracy'
)

lr_models = dict()

for model_name in embedding_models:
    X_train = X_trains[model_name]
    grid_search = grid_search.fit(X_train, y_train)
    lr_models[model_name] = grid_search.best_estimator_

In [28]:
for estimator_name in lr_models:
    X_test = X_tests[estimator_name]
    print(estimator_name)
    y_pred = lr_models[estimator_name].predict(X_test)
    display(classification_report_pd(y_test, y_pred))
    display(confusion_matrix(y_test, y_pred))
    print('=============================================')

cbow


,precision,recall,f1-score,support
0,0.833333,1.000000,0.909091,500
1,0.000000,0.000000,0.000000,100
accuracy,NaN,NaN,0.833333,600
macro avg,0.416667,0.500000,0.454545,600
weighted avg,0.694444,0.833333,0.757576,600


array([[500,   0],
       [100,   0]])

skipgram


,precision,recall,f1-score,support
0,0.954198,1.00,0.976562,500
1,1.000000,0.76,0.863636,100
accuracy,NaN,NaN,0.960000,600
macro avg,0.977099,0.88,0.920099,600
weighted avg,0.961832,0.96,0.957741,600


array([[500,   0],
       [ 24,  76]])

fasttext


,precision,recall,f1-score,support
0,0.982249,0.996000,0.989076,500
1,0.978495,0.910000,0.943005,100
accuracy,NaN,NaN,0.981667,600
macro avg,0.980372,0.953000,0.966041,600
weighted avg,0.981623,0.981667,0.981398,600


array([[498,   2],
       [  9,  91]])

Очень интересно и странно, изначально, когда я решил отказаться от идеи с классификацией постов с ленты, получалось примерно то же самое, все классифицировалось в один класс. В данном случае, с fastText получилось исправить ситуацию, но не с cbow/skipgram. 

При этом, skipgram относительно cbow хотя бы что-то обнаружил, то есть я был прав, когда говорил про то, что skipgram лучше. 

Не знаю, как сделать так, чтобы и они работали. Но в результате, получился достаточно хороший классификатор (последний).

  